In [32]:
import pandas as pd

file = pd.read_csv('./Input/data/no_mask/no_mask.txt')
file.head()

,user_id\toriginal_image\tface_id\tage\tgender\tx\ty\tdx\tdy\ttilt_ang\tfiducial_yaw_angle\tfiducial_score
30601258@N03\t10399646885_67c7d20df9_o.jpg\t1\t(25,32)\tf\t0\t414\t1086\t1383\t-115\t30\t17
30601258@N03\t10424815813_e94629b1ec_o.jpg\t2\t(25,32)\tm\t301\t105\t640\t641\t0\t0\t94
30601258@N03\t10437979845_5985be4b26_o.jpg\t1\t(25,32)\tf\t2395\t876\t771\t771\t175\t-30\t74
30601258@N03\t10437979845_5985be4b26_o.jpg\t3\t(25,32)\tm\t752\t1255\t484\t485\t180\t0\t47
30601258@N03\t11816644924_075c3d8d59_o.jpg\t2\t(25,32)\tm\t175\t80\t769\t768\t-75\t0\t34


In [37]:
import pandas as pd
import csv

file = pd.read_csv('./Input/data/no_mask/no_mask.txt', delimiter = '\t')
file.head()

,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score
0,30601258@N03,10399646885_67c7d20df9_o.jpg,1,"(25, 32)",f,0,414,1086,1383,-115,30,17
1,30601258@N03,10424815813_e94629b1ec_o.jpg,2,"(25, 32)",m,301,105,640,641,0,0,94
2,30601258@N03,10437979845_5985be4b26_o.jpg,1,"(25, 32)",f,2395,876,771,771,175,-30,74
3,30601258@N03,10437979845_5985be4b26_o.jpg,3,"(25, 32)",m,752,1255,484,485,180,0,47
4,30601258@N03,11816644924_075c3d8d59_o.jpg,2,"(25, 32)",m,175,80,769,768,-75,0,34


In [38]:
file.to_csv('no_mask.csv')

In [39]:
file = file.loc[:,['gender','age']]

In [269]:
over_60 = file['age'] == '(60, 100)'
is_male = file['gender'] == 'm' 
is_female = file['gender'] == 'f'
sub_file = file[over_60 & (is_male | is_female)]

In [284]:
sub_file.index

Int64Index([  744,   746,   752,   769,   798,   799,   805,   820,   829,
              831,
            ...
            21410, 21501, 21530, 21951, 21968, 22016, 22019, 22020, 22024,
            22034],
           dtype='int64', length=928)

In [285]:
sub_file['state'] = ['normal' for i in sub_file.index]

<ipython-input-285-3beef86b30df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_file['state'] = ['normal' for i in sub_file.index]


In [286]:
sub_file.sample(10)

,gender,age,state
19752,f,"(60, 100)",normal
19802,m,"(60, 100)",normal
16910,m,"(60, 100)",normal
3931,f,"(60, 100)",normal
4671,m,"(60, 100)",normal
6682,f,"(60, 100)",normal
6842,f,"(60, 100)",normal
19770,m,"(60, 100)",normal
10051,m,"(60, 100)",normal
3921,f,"(60, 100)",normal


In [287]:
sub_file.count()

gender    928
age       928
state     928
dtype: int64

In [288]:
sub_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 928 entries, 744 to 22034
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gender  928 non-null    object
 1   age     928 non-null    object
 2   state   928 non-null    object
dtypes: object(3)
memory usage: 29.0+ KB


In [289]:
sub_file.describe()

,gender,age,state
count,928,928,928
unique,2,1,1
top,f,"(60, 100)",normal
freq,466,928,928


In [ ]:
def labeling(df, folder_path):
    temp = pd.DataFrame()
    
    path_list = []
    full_path_list = [] 
    state_list = []
    for path in df["path"]:
        #사람 폴더에서 이미지 경로 받기.
        imgs = glob.glob(folder_path + "/" + path + "/*")
        
        path_list.extend([path]*7)
        full_path_list.extend([full_path for full_path in imgs])
        state_list.extend([full_path.split("/")[-1].split(".")[0] for full_path in imgs])
  
    temp["path"] = path_list
    temp["img_path"] = full_path_list
    temp["state"] = state_list
    return temp

# 기존 df와 생성된 df 합치기
def merge_df(df, mask_state_df):
    return_df = pd.merge(df, mask_state_df, how= "outer", on = "path")
    # print(return_df)
    return_df.loc[return_df.query("state == 'incorrect_mask'").index, "state"] = "incorrect"

    # 라벨링
    return_df["label"] = 0
    return_df.loc[return_df.query("state == 'incorrect'").index, "label"] += 6
    return_df.loc[return_df.query("state == 'normal'").index, "label"] += 12

    return_df.loc[return_df.query("gender == 'female'").index, "label"] += 3

    return_df.loc[return_df.query("30 <= age < 60 ").index, "label"] += 1
    return_df.loc[return_df.query("age == 60 ").index, "label"] += 2


    # valid_df = return_df.iloc[200:]
    # return_df = return_df.iloc[:200]

    return_df["c_gender"] = 0
    return_df.loc[return_df.query("gender == 'male'").index, "c_gender"] = 1

    return_df["c_state"] = 0
    return_df.loc[return_df.query("state == 'incorrect'").index, "c_state"] = 1
    return_df.loc[return_df.query("state == 'normal'").index, "c_state"] = 2

    return_df["c_age"] = 0
    return_df.loc[return_df.query("30 <= age < 60 ").index, "c_age"] = 1
    return_df.loc[return_df.query("age == 60 ").index, "c_age"] = 2
    
    return_df["c_age_deep"] = 0
    return_df.loc[return_df.query("20 <= age < 25 ").index, "c_age_deep"] = 1
    return_df.loc[return_df.query("25 <= age < 30 ").index, "c_age_deep"] = 2
    return_df.loc[return_df.query("30 <= age < 35 ").index, "c_age_deep"] = 3
    return_df.loc[return_df.query("35 <= age < 40 ").index, "c_age_deep"] = 4
    return_df.loc[return_df.query("40 <= age < 45 ").index, "c_age_deep"] = 5
    return_df.loc[return_df.query("45 <= age < 50 ").index, "c_age_deep"] = 6
    return_df.loc[return_df.query("50 <= age < 55 ").index, "c_age_deep"] = 7
    return_df.loc[return_df.query("55 <= age < 60 ").index, "c_age_deep"] = 8
    return_df.loc[return_df.query("age == 60 ").index, "c_age_deep"] = 9

    
    
    label_dict = {0: "마스크쓴 남성 청년",
              1: "마스크쓴 남성 중년",
              2: "마스크쓴 남성 노년",
              3: "마스크쓴 여성 청년",
              4: "마스크쓴 여성 중년",
              5: "마스크쓴 여성 노년",
              
              6: "잘못쓴 남성 청년",
              7: "잘못쓴 남성 중년",
              8: "잘못쓴 남성 노년",
              9: "잘못쓴 여성 청년",
              10: "잘못쓴 여성 중년",
              11: "잘못쓴 여성 노년",
              
              12: "안쓴 남성 청년",
              13: "안쓴 남성 중년",
              14: "안쓴 남성 노년",
              15: "안쓴 여성 청년",
              16: "안쓴 여성 중년",
              17: "안쓴 여성 노년",
    }
    return_df["label_check"] = return_df["label"].replace(label_dict)
    return return_df


mask_state_train_df = labeling(train_df, "Input/data/no_mask/images")  
mask_state_valid_df = labeling(valid_df, "Input/data/no_mask/images")

train_df = merge_df(train_df, mask_state_train_df)
valid_df = merge_df(valid_df, mask_state_valid_df)

valid_df.sample(20)[["gender","age","state","label_check"]]